# Build RAG chatbot using cohere 

## First: Install dependencies

In [1]:
from dotenv import load_dotenv
load_dotenv()

import cohere
import os
import hnswlib
import json
import uuid
from typing import List, Dict, Dict
from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title

co = cohere.Client(os.environ["COHERE_API_KEY"])